## Building the dataset that will be input into the model

In [2]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("Building dataset").config("hive.metastore.uris", "thrift://amok:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session
spark.sql("USE 2023_11_02")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hadoop-3.2.2/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2024-01-07 16:07:25,142 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-07 16:07:26,326 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2024-01-07 16:07:28,214 WARN spark.ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental f

DataFrame[]

In [3]:
#import pyspark.pandas as ps
from pyspark.sql.functions import lit,col
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
#from boruta import BorutaPy
#from fredapi import Fred
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
import csv
from pyspark.sql import functions as F
from functools import reduce
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from CreateDataset import get_features_all_stocks_seq, get_full_series_stocks

In [4]:
df = pd.read_csv('imploded_stocks_price.csv', index_col=False)
#full_series_stocks = get_full_series_stocks(df) #this gets the stocks that have data since 2001
#filtered_df = df[df['fsym_id'].isin(full_series_stocks)]
df['Implosion_Start_Date'] = pd.to_datetime(df['Implosion_Start_Date'])
df['Implosion_End_Date'] = pd.to_datetime(df['Implosion_End_Date'])
df = get_features_all_stocks_seq(df, all_feats=False) #get stocks that have data in the ff_advanced_der_af for all years, not just prices (or not even)
#for boruta, maybe treat each column separately 22*10 features?
#df.show()
print(df.count())


/opt/spark/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/spark/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
2024-01-07 16:08:23,814 WARN scheduler.TaskSetManager: Stage 1 contains a task of very large size (2960 KiB). The maximum recommended task size is 1000 KiB.


4482


In [5]:
print(df.filter(df['label'] == 1).count())

2024-01-07 16:09:33,376 WARN scheduler.TaskSetManager: Stage 13 contains a task of very large size (2960 KiB). The maximum recommended task size is 1000 KiB.


272


In [6]:
df.show(5)

2024-01-07 16:09:49,805 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
2024-01-07 16:09:51,204 WARN scheduler.TaskSetManager: Stage 24 contains a task of very large size (2960 KiB). The maximum recommended task size is 1000 KiB.


+--------+----------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
| fsym_id|ff_net_inc_bef_xord_gr|     ff_capex_assets|     ff_div_yld_secs|             ff_roic|         ff_earn_yld|ff_debt_st_x_curr_port|             ff_roce|      ff_oper_inc_gr|    ff_reinvest_rate|            ff_wkcap|  ff_commiss_inc_net| ff_oper_cf_fix_chrg|  ff_debt_entrpr_val|ff_net_inc_dil_bef_unusual|             ff_xord|        ff_xord_disc|      ff_liabs_lease|     ff_pfd_stk_tcap|        

In [8]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def train_test_split(df):
    train, test = df.randomSplit([0.7,0.3], 22)
    return train, test


def confusion_matrix_pandas(df):
    df = df.toPandas()
    cm = confusion_matrix(df['label'], df['prediction'])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
    plt.title(f'Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    



def model_testing(df):
    df = df.filter(reduce(lambda acc, column: acc & (F.size(col(column)) == 22), df.columns[1:-1], lit(True)))

    #need to decide whether to only include stocks that started from 2000, or include just from e.g. 2019
    #temporary measure - replace with 0
    #try imputer?
    #look into masking
    
    print("Number of records: ", df.count())
    features = df.columns[1:-1]
    list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
    for f in features:
        df = df.withColumn(f, list_to_vector_udf(f))
    df.show(2)
    train_df, test_df = train_test_split(df)
    
    vector_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")
    train_df = vector_assembler.transform(train_df)
    test_df = vector_assembler.transform(test_df)
    
    lr = LogisticRegression(featuresCol="features_vector", labelCol="label")
    rf = RandomForestClassifier(featuresCol='features_vector', labelCol='label')
    gbt = GBTClassifier(featuresCol = 'features_vector', labelCol='label')
    models = [rf]
    model_names = ['random forest']
    
    for model, model_name in zip(models, model_names):
        if model_name == 'random forest':
            paramGrid = ParamGridBuilder() \
                .addGrid(model.numTrees, [100]) \
                .addGrid(model.maxDepth, [15]) \
                .addGrid(model.minInstancesPerNode, [5]) \
                .build()
        elif model_name == 'logistic regression':
            paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1]) \
                .addGrid(lr.elasticNetParam, [0.5]) \
                .addGrid(lr.maxIter, [100]) \
                .build()
        else:
            paramGrid = ParamGridBuilder() \
                .addGrid(gbt.maxDepth, [10]) \
                .addGrid(gbt.maxBins, [32]) \
                .addGrid(gbt.maxIter, [20]) \
                .build()

        evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

        crossval = CrossValidator(estimator=model,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=evaluator,
                                  numFolds=5, parallelism=12)

        cvModel = crossval.fit(train_df)

        avg_metrics = cvModel.avgMetrics
        print(model_name.upper())

        for i, acc in enumerate(avg_metrics):
            print(f"Fold {i + 1} - Validation Accuracy: {acc}")

        best_model = cvModel.bestModel

        predictions = best_model.transform(test_df)

        confusion_matrix_pandas(predictions.select('label', 'prediction'))
        
def basic_test(df):
    df = df.filter(reduce(lambda acc, column: acc & (F.size(col(column)) == 22), df.columns[1:-1], lit(True)))
    print("Number of records: ", df.count())
    features = df.columns[1:-1]
    list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
    for f in features:
        df = df.withColumn(f, list_to_vector_udf(f))
    df.show(2)
    train_df, test_df = train_test_split(df)
    
    vector_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")
    train_df = vector_assembler.transform(train_df)
    test_df = vector_assembler.transform(test_df)
    
    lr = LogisticRegression(featuresCol="features_vector", labelCol="label")
    rf = RandomForestClassifier(featuresCol='features_vector', labelCol='label')
    gbt = GBTClassifier(featuresCol = 'features_vector', labelCol='label')
    models = [lr, rf, gbt]
    model_names = ['logistic regression']
    
    for model, model_name in zip(models, model_names):
        if model_name == 'boosted trees':
            paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [10])
             .addGrid(gbt.maxIter, [50])
             .build())
        elif model_name == 'random forest':
            paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [5]) #5,10,15
             .addGrid(rf.numTrees, [20]) #20,50,100
             .build())
        else:
            paramGrid = (ParamGridBuilder()
             .addGrid(lr.maxIter, [10])  # Number of iterations #10,50,100
             .addGrid(lr.regParam, [0.01])  # Regularization parameter #0.01,0.1,0.5
             .addGrid(lr.elasticNetParam, [0.0])  # Elastic net parameter (0 for L2, 1 for L1) 0.0,0.5,1.0
             .build())
    


        evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
        
        crossval = CrossValidator(estimator = model, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
        
        model = crossval.fit(train_df)
        
        best_model = model.bestModel

        print(model_name.upper())

        predictions = best_model.transform(test_df)

        confusion_matrix_pandas(predictions.select('label', 'prediction'))
        
        recall = evaluator.evaluate(predictions)
        print(f"Recall: {recall}")
    


#basic_test(df)
#test_pandas(df)
#model_testing(df)

In [11]:
from hyperopt import fmin, tpe, hp
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics 
    
def model_training(df, classifier):
    df = df.filter(reduce(lambda acc, column: acc & (F.size(col(column)) == 22), df.columns[1:-1], lit(True)))
    print("Number of records: ", df.count())
    
    features = df.columns[1:-1]

    list_to_vector_udf = F.udf(lambda l: Vectors.dense(l), VectorUDT())
    for f in features:
        df = df.withColumn(f, list_to_vector_udf(f))
 
    train_df, test_df = train_test_split(df)

    vector_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")
    train_df = vector_assembler.transform(train_df)
    test_df = vector_assembler.transform(test_df)

    if classifier == 'LogisticRegression':
        param_space = {
            'regParam': hp.uniform('regParam', 0.01, 1.0),
            'elasticNetParam': hp.uniform('elasticNetParam', 0.0, 1.0)
        }
        classifier_instance = LogisticRegression(featuresCol="features_vector", labelCol="label")
    elif classifier == 'RandomForest':
        param_space = {
            'numTrees': hp.choice('numTrees', [10, 20, 30]),
            'maxDepth': hp.choice('maxDepth', [5, 10, 15])
        }
        classifier_instance = RandomForestClassifier(featuresCol='features_vector', labelCol='label', numTrees=10)
    elif classifier == 'GBT':
        param_space = {
            'maxDepth': hp.choice('maxDepth', [5, 10, 15]),
            'maxIter': hp.choice('maxIter', [10, 20, 30])
        }
        classifier_instance = GBTClassifier(featuresCol='features_vector', labelCol='label')
    else:
        raise ValueError("Unsupported classifier")

    def objective(params):
        classifier_instance.setParams(**params)
        #evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
        evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', metricName='recallByLabel')
        crossval = CrossValidator(estimator=classifier_instance,
                                  estimatorParamMaps=ParamGridBuilder().build(),
                                  evaluator=evaluator,
                                  numFolds=3)
        
        cv_model = crossval.fit(train_df)
        predictions = cv_model.transform(train_df)
        val_metric = evaluator.evaluate(predictions)
        return val_metric

    # Find the best hyperparameters
    best_params = fmin(fn=objective, space=param_space, algo=tpe.suggest, max_evals=5)
    print("Best hyperparameters: ", best_params)

    # Train the model with the best hyperparameters
    classifier_instance.setParams(**best_params)
    best_model = classifier_instance.fit(train_df)
    feature_importances = best_model.featureImportances
    print("Feature Importances:")
    for i in range(len(feature_importances)):
        print("Feature {}: {}".format(i, feature_importances[i]))
        
    predictions = best_model.transform(test_df)

    confusion_matrix_pandas(predictions.select('label', 'prediction'))
    
    
    
model_training(df, 'GBT')

2024-01-07 17:22:12,280 WARN scheduler.TaskSetManager: Stage 4264 contains a task of very large size (2960 KiB). The maximum recommended task size is 1000 KiB.


Number of records:  4475


2024-01-07 17:22:38,769 WARN scheduler.TaskSetManager: Stage 4277 contains a task of very large size (2960 KiB). The maximum recommended task size is 1000 KiB.
2024-01-07 17:23:03,517 WARN scheduler.TaskSetManager: Stage 4290 contains a task of very large size (2960 KiB). The maximum recommended task size is 1000 KiB.


  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

2024-01-07 17:23:28,575 WARN scheduler.TaskSetManager: Stage 4304 contains a task of very large size (2960 KiB). The maximum recommended task size is 1000 KiB.
2024-01-07 17:38:17,139 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1000.8 KiB
2024-01-07 17:38:32,215 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1001.6 KiB
2024-01-07 17:38:35,492 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1003.8 KiB
2024-01-07 17:38:39,064 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1006.1 KiB
2024-01-07 17:38:42,227 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1007.9 KiB
2024-01-07 17:38:44,958 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1009.2 KiB
2024-01-07 17:38:47,703 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1010.6 KiB
2024-01-07 17:38:50,332 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1011.9 KiB


 20%|██        | 1/5 [1:36:40<6:26:40, 5800.20s/trial, best loss: 0.9456967213114754]

2024-01-07 19:00:08,717 WARN scheduler.TaskSetManager: Stage 22391 contains a task of very large size (2960 KiB). The maximum recommended task size is 1000 KiB.


 20%|██        | 1/5 [1:38:19<6:33:16, 5899.19s/trial, best loss: 0.9456967213114754]


KeyboardInterrupt: 

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

def train_tree(train_df):
    rf = RandomForestClassifier(featuresCol='features_vector', labelCol='label', maxDepth=maxDepth, numTrees=numTrees)
    model = rf.fit(train_df)
    preds = model.transform()
    

            
    
    

def train_with_hyperopt(params, df):
    numTrees = int(params['numTrees'])
    maxDepth = int(params['maxDepth'])

    model, f1_score = train_rf(numTrees, maxDepth, df)
    loss = - f1_score
    return {'loss': loss, 'status': STATUS_OK}

def prepare_df():
    df = df.filter(reduce(lambda acc, column: acc & (F.size(col(column)) == 22), df.columns[1:-1], lit(True)))
    print("Number of records: ", df.count())
    features = df.columns[1:-1]
    list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
    for f in features:
        df = df.withColumn(f, list_to_vector_udf(f))
    df.show(2)
    train_df, test_df = train_test_split(df)
    
    vector_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")
    train_df = vector_assembler.transform(train_df)
    test_df = vector_assembler.transform(test_df)
    return train_df, test_df

# train_df, test_df = prepare_df(df)
# train_with_hyperopt()


In [ ]:
#print(filtered_df.filter(F.col('label')==1).count())

In [ ]:
#df.filter(F.col('label')==1).count()

In [ ]:
spark.stop()